In [15]:
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory

from langchain_google_genai import GoogleGenerativeAI
from langchain_google_community import VertexAISearchRetriever

from dotenv import load_dotenv
import os
# Load environment variables
load_dotenv()

True

In [16]:
# Retriever

retriever = VertexAISearchRetriever(
    project_id="gen-lang-client-0647812146",
    location_id="global",
    data_store_id="legalaco_1746207105123",
    max_documents=5
)

/home/wlopez/rust/ia-generativa-cartera/rag-gemini/.venv/lib/python3.12/site-packages/langchain_google_community/vertex_ai_search.py:364: UserWarning: Beta features are configured but beta=False. The following beta features will be ignored:['custom_embedding_ratio']
  warnings.warn(
/home/wlopez/rust/ia-generativa-cartera/rag-gemini/.venv/lib/python3.12/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [17]:
result = retriever.invoke("De que trata el documento?")
print(result)

[Document(metadata={'id': '76d8fa4d8431ff949d9519d789b33c09', 'source': 'gs://legalaco/LeyGeneralDeContratacionesPublicas.pdf', 'previous_segments': [{'content': '# d) Inscripción o reinscripción de consultores de obras, persona jurídica extranjera\n\n1. Solicitud presentada mediante formulario. 2. Indicar el número de recibo y fecha del pago (por derecho de tramitación). 3. Copia de la escritura pública inscrita en Registros Públicos o ante autoridad competente en su país de origen o copia del documento oficial emitido por la autoridad registral, institución o autoridad competente en su país de origen, que acredite que su representante legal cuenta con la facultad de representación en procedimientos administrativos ante entidades públicas fuera de su país de origen, en la cual debe encontrarse comprendido el Perú, cuando la persona jurídica actúe con representante legal desde su país de origen. 4. Copia de documento oficial emitido por autoridad registral, institución o autoridad comp

In [18]:
llm = GoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0.5,
    top_p=0.8,
    top_k=40,
    google_api_key=os.getenv("GOOGLE_API_KEY")
)

In [19]:
llm.invoke("hola como estas?")

'¡Hola! Estoy bien, gracias por preguntar. ¿Y tú, cómo estás? ¿En qué puedo ayudarte hoy?'

In [20]:
#Template

template = """
  Use the context and the history to answer the question.
  If do not have enough information to answer the question, say: "I do not have enough information".
  You allways answer in Spanish.
  You are a lawyer and you are answering a question about a legal document.

  Context:
  {context}

  History:
  {history}

  Question:
  {question}

  Answer:
"""

prompt = PromptTemplate(
    template=template,
    input_variables=[
        "history",
        "context",
        "question"
    ]
)
  

In [21]:
#Q&A

qa = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="stuff",
    chain_type_kwargs={
                "verbose": True,
                "prompt": prompt,
                "memory": ConversationBufferMemory(memory_key="history", input_key="question")
            })

/tmp/ipykernel_297548/1729249497.py:10: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  "memory": ConversationBufferMemory(memory_key="history", input_key="question")


In [22]:

x = qa({"query": "de que trata el documento?"})
# Print the result
print(x)

/tmp/ipykernel_297548/3308726628.py:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  x = qa({"query": "de que trata el documento?"})




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

  Use the context and the history to answer the question.
  If do not have enough information to answer the question, say: "I do not have enough information".
  You allways answer in Spanish.
  You are a lawyer and you are answering a question about a legal document.

  Context:
  Situacionales conteniendo como mínimo el Estado de Situación Financiera, Estado de Resultados y las Notas Contables respectivas, debidamente firmados por el órgano de administración de la matriz, a fin de demostrar haber restablecido el equilibrio económico entre el capital y el patrimonio neto para cumplir con los parámetros de medición de solvencia económica establecidos para el RNP. 7. De contar con experiencia en consultoría de obras culminadas en el Perú y/o en el extranjero, se debe acreditar con lo siguiente: 7.1 Para solicitud de inscripción y/o reinscripción i. Consultoría de obras culminadas en

## ¿Cómo saber en qué página se encuentra la respuesta?
Cuando usas un retriever como `VertexAISearchRetriever`, los documentos recuperados suelen incluir metadata, como el identificador del documento, la fuente y, en algunos casos, la página de donde proviene el fragmento. Puedes acceder a esta información desde el resultado del retriever o de la cadena QA.

In [26]:
# Ejemplo: Mostrar metadata de los documentos recuperados (incluyendo página si está disponible)
docs = retriever.invoke("De que trata el documento?")
print(docs)

[Document(metadata={'id': '76d8fa4d8431ff949d9519d789b33c09', 'source': 'gs://legalaco/LeyGeneralDeContratacionesPublicas.pdf', 'previous_segments': [{'content': '# d) Inscripción o reinscripción de consultores de obras, persona jurídica extranjera\n\n1. Solicitud presentada mediante formulario. 2. Indicar el número de recibo y fecha del pago (por derecho de tramitación). 3. Copia de la escritura pública inscrita en Registros Públicos o ante autoridad competente en su país de origen o copia del documento oficial emitido por la autoridad registral, institución o autoridad competente en su país de origen, que acredite que su representante legal cuenta con la facultad de representación en procedimientos administrativos ante entidades públicas fuera de su país de origen, en la cual debe encontrarse comprendido el Perú, cuando la persona jurídica actúe con representante legal desde su país de origen. 4. Copia de documento oficial emitido por autoridad registral, institución o autoridad comp

### Cómo extraer la información de página de los documentos recuperados
Puedes acceder a la metadata de cada documento, incluyendo información de páginas si está disponible, recorriendo la lista de documentos y accediendo a los campos relevantes en `metadata`. Por ejemplo, si existe un campo `previous_segments` o `next_segments` con información de páginas, puedes extraerla así:

In [27]:
# Mostrar información de páginas si está disponible en los segmentos
for doc in docs:
    print('ID:', doc.metadata.get('id'))
    print('Fuente:', doc.metadata.get('source'))
    # Buscar información de páginas en previous_segments
    prev_segments = doc.metadata.get('previous_segments', [])
    for seg in prev_segments:
        page_span = seg.get('page_span')
        if page_span:
            print('Páginas (segmento previo):', page_span)
    # Buscar información de páginas en next_segments
    next_segments = doc.metadata.get('next_segments', [])
    for seg in next_segments:
        page_span = seg.get('page_span')
        if page_span:
            print('Páginas (segmento siguiente):', page_span)
    print('---')

ID: 76d8fa4d8431ff949d9519d789b33c09
Fuente: gs://legalaco/LeyGeneralDeContratacionesPublicas.pdf
Páginas (segmento previo): {'page_end': 255.0, 'page_start': 251.0}
Páginas (segmento siguiente): {'page_end': 254.0, 'page_start': 253.0}
---


### Explicación de los campos de metadata
- **ID:** Es el identificador único del documento dentro del sistema o base de datos.
- **Fuente:** Es la ubicación o ruta del archivo fuente, en este caso un PDF almacenado en Google Cloud Storage.
- **Páginas (segmento previo):** Indica el rango de páginas del segmento anterior relacionado con el contenido recuperado. Por ejemplo, `{'page_end': 255.0, 'page_start': 251.0}` significa que el segmento previo abarca desde la página 251 hasta la 255 del documento.
- **Páginas (segmento siguiente):** Indica el rango de páginas del segmento siguiente relacionado con el contenido recuperado. Por ejemplo, `{'page_end': 254.0, 'page_start': 253.0}` significa que el segmento siguiente abarca desde la página 253 hasta la 254 del documento.

Estos campos te permiten saber de qué documento y de qué páginas proviene la información utilizada para responder la pregunta.